## Import Libraries

In [62]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime as dt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

## Create dictionary to hold months and associated sales that will be used for forecasting

In [63]:
#dictionary to hold dataset 
month_sales ={"Month": ["January 2023","February 2023","March 2023","April 2023","May 2023","June 2023", "July 2023","August 2023","September 2023","October 2023","November 2023","December 2023"],
              "Sales": [74,66,59,66,68,61,57,60,69,71,59,70]}

In [64]:
#turn dictionary into dataframe 
df = pd.DataFrame(month_sales)
df

,Month,Sales
0,January 2023,74
1,February 2023,66
2,March 2023,59
3,April 2023,66
4,May 2023,68
5,June 2023,61
6,July 2023,57
7,August 2023,60
8,September 2023,69
9,October 2023,71


In [65]:
#convert the month into datetime object
df["Month"] = pd.to_datetime(df["Month"],format='%B %Y' )
#print dtypes of df
print(df.dtypes)

#set month as the index of the dataframe
df.set_index("Month",inplace=True)
df

Month    datetime64[ns]
Sales             int64
dtype: object


,Sales
Month,
2023-01-01,74
2023-02-01,66
2023-03-01,59
2023-04-01,66
2023-05-01,68
2023-06-01,61
2023-07-01,57
2023-08-01,60
2023-09-01,69


## Naive Approach

In [66]:
#initialize new column in df for naive approach
df["Naive"]= None
previous_sales = None

#manually iterate through sales column of df and apped last period (month) value to new column
for idx, row in df.iterrows():
    df.at[idx,"Naive"] = previous_sales
    previous_sales = row["Sales"]

df

,Sales,Naive
Month,,
2023-01-01,74,None
2023-02-01,66,74
2023-03-01,59,66
2023-04-01,66,59
2023-05-01,68,66
2023-06-01,61,68
2023-07-01,57,61
2023-08-01,60,57
2023-09-01,69,60


## Moving Average

In [67]:
#slice the dataframe using iloc
#moving_df = df.iloc[5:]

moving_df = df.loc["2023-04":]
moving_df

,Sales,Naive
Month,,
2023-04-01,66,59
2023-05-01,68,66
2023-06-01,61,68
2023-07-01,57,61
2023-08-01,60,57
2023-09-01,69,60
2023-10-01,71,69
2023-11-01,59,71
2023-12-01,70,59


In [68]:
moving_df["Moving Average"]=moving_df["Sales"].rolling(window=3).mean()
moving_df

C:\Users\mburd\AppData\Local\Temp\ipykernel_27916\1257938677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moving_df["Moving Average"]=moving_df["Sales"].rolling(window=3).mean()


,Sales,Naive,Moving Average
Month,,,
2023-04-01,66,59,NaN
2023-05-01,68,66,NaN
2023-06-01,61,68,65.000000
2023-07-01,57,61,62.000000
2023-08-01,60,57,59.333333
2023-09-01,69,60,62.000000
2023-10-01,71,69,66.666667
2023-11-01,59,71,66.333333
2023-12-01,70,59,66.666667


### Weighted Moving Average

In [69]:
#define a numpy array to hold the weight values 
weights = np.array([0.1,0.3,0.6])

#weighted moving average calculation 
moving_df["Weighted Moving Average"] = moving_df["Sales"].rolling(window=len(weights)).apply(lambda x: np.dot(x,weights),raw=True)
moving_df

C:\Users\mburd\AppData\Local\Temp\ipykernel_27916\179554011.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moving_df["Weighted Moving Average"] = moving_df["Sales"].rolling(window=len(weights)).apply(lambda x: np.dot(x,weights),raw=True)


,Sales,Naive,Moving Average,Weighted Moving Average
Month,,,,
2023-04-01,66,59,NaN,NaN
2023-05-01,68,66,NaN,NaN
2023-06-01,61,68,65.000000,63.6
2023-07-01,57,61,62.000000,59.3
2023-08-01,60,57,59.333333,59.2
2023-09-01,69,60,62.000000,65.1
2023-10-01,71,69,66.666667,69.3
2023-11-01,59,71,66.333333,63.6
2023-12-01,70,59,66.666667,66.8


## Exponential Smoothing

In [70]:
#apply exponential smoothing

model = SimpleExpSmoothing(moving_df['Sales'])
fit = model.fit(smoothing_level=0.2,optimized=False)
moving_df["Exponential Smoothing"]=fit.fittedvalues

moving_df

C:\Users\mburd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\mburd\AppData\Local\Temp\ipykernel_27916\2228421487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moving_df["Exponential Smoothing"]=fit.fittedvalues


,Sales,Naive,Moving Average,Weighted Moving Average,Exponential Smoothing
Month,,,,,
2023-04-01,66,59,NaN,NaN,66.000000
2023-05-01,68,66,NaN,NaN,66.000000
2023-06-01,61,68,65.000000,63.6,66.400000
2023-07-01,57,61,62.000000,59.3,65.320000
2023-08-01,60,57,59.333333,59.2,63.656000
2023-09-01,69,60,62.000000,65.1,62.924800
2023-10-01,71,69,66.666667,69.3,64.139840
2023-11-01,59,71,66.333333,63.6,65.511872
2023-12-01,70,59,66.666667,66.8,64.209498


## Forecasting Error 

### Calculate Mean Absolute Deviation

In [71]:
# create function that manually calculates MAD
def mean_absolute_deviation(actual_value,predicted_value):
    mad = abs(actual_value - predicted_value).mean()
    print(f"The MAD of this forecasting method is {mad}")

### Calculate Mean Absolute Percentage Error

In [72]:
#create function that manually calcualtes MAPE
def mean_absolute_percentage_error(actual_value, predicted_value):
    mape = (abs((actual_value-predicted_value)/actual_value).mean()) * 100
    print(f"The MAPE of this forecasting method is {mape}%")

### Calculate Mean Squared Error

In [73]:
#create a fucntion that manually calculates mse 
def mean_squared_error(actual_value,predicted_value):
    mse = ((actual_value-predicted_value) ** 2).mean() 
    print(f"The MSE of this forecasting method is {mse}")

In [74]:
#create list that will hold df columns with different forecasting method series
#create variable to hold actual values of sales from moving_df
forecasting_outputs = {
    "Moving Average": moving_df["Moving Average"],
    "Weighted Moving Average": moving_df["Weighted Moving Average"],
    "Exponential Smoothing": moving_df["Exponential Smoothing"]
}
forecasting_actuals = moving_df["Sales"]

## Print results to See the various errors for each kind of forecasting method

In [75]:
#iterate through forecasting outputs list 
for name,column in forecasting_outputs.items():
    print(name)
    mean_absolute_deviation(forecasting_actuals,column)
    mean_absolute_percentage_error(forecasting_actuals,column)
    mean_squared_error(forecasting_actuals,column)

Moving Average
The MAD of this forecasting method is 4.523809523809521
The MAPE of this forecasting method is 7.125702171291348%
The MSE of this forecasting method is 24.873015873015852
Weighted Moving Average
The MAD of this forecasting method is 2.728571428571431
The MAPE of this forecasting method is 4.292184997963642%
The MSE of this forecasting method is 8.884285714285728
Exponential Smoothing
The MAD of this forecasting method is 4.957081600000001
The MAPE of this forecasting method is 7.806612487199799%
The MSE of this forecasting method is 30.62810902826555
